In [1]:
%load_ext autoreload
%autoreload 2

In [78]:
import os
import glob
from honeybee_energy import run as hbe
from eppy.modeleditor import IDF
import sys
iddfile = "/Applications/EnergyPlus-22-2-0/Energy+.idd"
IDF.setiddname(iddfile)

'darwin'

In [3]:
vintage = 'pre1980'
cz = '1a'

In [23]:
project_root = '/Users/jmccarty/GitHub/cmip6_and_buildings'
# idf_dir = glob.glob(os.path.join(project_root,'eplus_simulations',vintage,'idf_files','v22_2',f'{cz}*'))[0]
idf_path = "/Users/jmccarty/Desktop/test_idf_run/test_idf.idf"
epw = glob.glob(os.path.join(project_root,'weather_files','tmy3',f'{cz}*'))[0]
# hbe.run_idf(idf_file_path=idf, epw_file_path=epw, )



['/Users/jmccarty/GitHub/cmip6_and_buildings/',
 '/pre1980/idf_files/v22_2/1a_pre1980_v22_2/RefBldgFullServiceRestaurantPre1980_V22-2-0_1A_USA_FL_MIAMI.idf']

In [46]:
idf_file = "/Users/jmccarty/GitHub/cmip6_and_buildings/eplus_simulations/pre1980/idf_files/v22_2/1a_pre1980_v22_2/RefBldgFullServiceRestaurantPre1980_V22-2-0_1A_USA_FL_MIAMI.idf"

def decomponse_idf_filename(idf_file):
    sim_dir_split = idf_file.split("eplus_simulations")
    file_segments = sim_dir_split[1].split(os.sep)
    idf_segments = file_segments[-1].split("_")

    return {'project_root': sim_dir_split[0],
            'vintage': file_segments[1],
            'version': file_segments[3],
            'building_type': idf_segments[0],
            'climate_zone': idf_segments[2].lower()}
    
decomponse_idf_filename(idf_file)

{'project_root': '/Users/jmccarty/GitHub/cmip6_and_buildings/',
 'vintage': 'pre1980',
 'version': 'v22_2',
 'building_type': 'RefBldgFullServiceRestaurantPre1980',
 'climate_zone': '1a'}

In [36]:
a = glob.glob(os.path.join(project_root,'eplus_simulations',vintage,'idf_files','v22_2',f'{cz}*'))[0]
# os.path.dirname(a)
idf_fname = glob.glob(os.path.join(a,'*.idf'))[0]
cz_dir = os.path.dirname(idf_fname).split(os.sep)[-1]

['',
 'Users',
 'jmccarty',
 'GitHub',
 'cmip6_and_buildings',
 'eplus_simulations',
 'pre1980',
 'idf_files',
 'v22_2',
 '1a_pre1980_v22_2']

In [57]:

def decompose_idf_filename(idf_file):
    sim_dir_split = idf_file.split("eplus_simulations")
    file_segments = sim_dir_split[1].split(os.sep)
    idf_segments = file_segments[-1].split("_")

    details = {'project_root': sim_dir_split[0],
            'vintage': file_segments[1],
            'version': file_segments[3],
            'building_type': idf_segments[0],
            'climate_zone': idf_segments[2].lower()
            }
    return details


def create_results_dir(idf):
    fname = idf.idfname
    idf_details = decompose_idf_filename(fname)

    results_pardir = os.path.join(idf_details['project_root'],
                                  'eplus_simulations',
                                  idf_details['vintage'],
                                  idf_details['version'],
                                  idf_details['climate_zone']
                                  )
    # check_dir_create(results_pardir)

    results_dir = os.path.join(results_pardir, idf_details['building_type'])
    # check_dir_create(results_dir)

    return results_dir

In [56]:
create_results_dir(idf)

/Users/jmccarty/Desktop/test_idf.idf


KeyError: 'vintage'

In [ ]:
def get_year_bands(year, formatted=True):
    if formatted:
        year_bands = f'{year-15}-{year+15}'
    else:
        year_bands = (year-15, year+15)
    return year_bands

def get_epw_name(idf_fname, scenario, year, ptile=50):
    idf_details = decompose_idf_filename(idf_path)
    project_root = idf_details['project_root']
    cz = idf_details['climate_zone']
    
    
    if scenario=='historical':
        epw_path = glob.glob(os.path.join(project_root,'weather_files','tmy3',f'{cz}*.epw'))[0]
    else:
        year_bands = get_year_bands(year, formatted=True)
        epw_path = os.path.join(project_root,'weather_files','morphed',cz,scenario,'EPWs',f'{scenario}_{ptile}_{year_bands}.epw')
    return epw_path

scenarios = ['historical', 'ssp126', 'ssp245', 'ssp585']
years = [2020, 2035, 2050, 2065, 2080]
for scenario in scenarios:
    for year in years:
        if scenario=='historical':
            year = 2020
        else:
            if year==2020:
                continue
        epw_path = get_epw_name(idf_path, scenario, year)
        print(epw_path, os.path.exists(epw_path))



In [97]:
project_root = "/Users/jmccarty/GitHub/cmip6_and_buildings"
vintages = ['pre1980','post1980', 'new']

def load_idf_list(vintages):
    all_idf = []
    for vintage in vintages:
        for par_dir in glob.glob(os.path.join(project_root, 'eplus_simulations', vintage, 'idf_files', 'v22_2','*v22_2')):
            idf_files = glob.glob(os.path.join(par_dir,'*.idf'))
            all_idf.append(idf_files)
    
    return [item for sublist in all_idf for item in sublist]


768

In [58]:
idf_path = "/Users/jmccarty/GitHub/cmip6_and_buildings/eplus_simulations/post1980/idf_files/v22_2/3a_post1980_v22_2/RefBldgPrimarySchoolPost1980_V22-2-0_3A_USA_GA_ATLANTA.idf"

idf = IDF(idf_path, epw)

idfversion = idf.idfobjects['version'][0].Version_Identifier.split('.')
idfversion.extend([0] * (3 - len(idfversion)))
idfversionstr = '-'.join([str(item) for item in idfversion])
fname = idf.idfname
options = {
    'ep_version': idfversionstr,  # runIDFs needs the version number
    'output_prefix': os.path.basename(fname).split('.')[0],
    'output_suffix': 'D',
    'output_directory': create_results_dir(idf),
    'readvars': True,
    'expandobjects': True
}
options

{'ep_version': '22-2-0',
 'output_prefix': 'RefBldgPrimarySchoolPost1980_V22-2-0_3A_USA_GA_ATLANTA',
 'output_suffix': 'C',
 'output_directory': '/Users/jmccarty/GitHub/cmip6_and_buildings/eplus_simulations/post1980/v22_2/3a/RefBldgPrimarySchoolPost1980',
 'readvars': True,
 'expandobjects': True}

In [29]:
results_path = os.path.join(os.path.dirname(idf_path),'results')
if os.path.exists(results_path):
    pass
else:
    os.mkdir(results_path)
os.chdir(results_path)

idf = IDF(idf_path, epw)
idf.run()


/Applications/EnergyPlus-22-2-0/energyplus --weather /Users/jmccarty/GitHub/cmip6_and_buildings/weather_files/tmy3/1a_USA_FL_Miami_Intl.epw --output-directory /Users/jmccarty/Desktop/test_idf_run/results --idd /Applications/EnergyPlus-22-2-0/Energy+.idd /Users/jmccarty/Desktop/test_idf_run/test_idf_79cceb.idf

EnergyPlus Starting
EnergyPlus, Version 22.2.0-c249759bad, YMD=2023.05.04 10:43
Initializing Response Factors
Calculating CTFs for "STEEL FRAME RES EXT WALL"
Calculating CTFs for "IEAD RES ROOF"
Calculating CTFs for "EXT-SLAB"
Calculating CTFs for "INT-WALLS"
Calculating CTFs for "INT-FLOOR-TOPSIDE"
Calculating CTFs for "INTERIORFURNISHINGS"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces

EnergyPlus Completed Successfully.


In [17]:
help(idf.run)

Help on method run in module eppy.modeleditor:

run(**kwargs) method of eppy.modeleditor.IDF instance
    This method wraps the following method:
    
    run(idf=None, weather=None, output_directory='', annual=False, design_day=False, idd=None, epmacro=False, expandobjects=False, readvars=False, output_prefix=None, output_suffix=None, version=False, verbose='v', ep_version=None)
        Wrapper around the EnergyPlus command line interface.
        
        Parameters
        ----------
        idf : str
            Full or relative path to the IDF file to be run, or an IDF object.
        
        weather : str
            Full or relative path to the weather file.
        
        output_directory : str, optional
            Full or relative path to an output directory (default: 'run_outputs)
        
        annual : bool, optional
            If True then force annual simulation (default: False)
        
        design_day : bool, optional
            Force design-day-only simulati

In [2]:
import eppy_multi

In [9]:
eppy_multi.main()

: 

: 